In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,10396
2,Huelva,Confirmados PDIA 14 días,985
3,Huelva,Tasa PDIA 14 días,"192,85628975825415"
4,Huelva,Confirmados PDIA 7 días,362
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,0
629,Municipio de Huelva sin especificar,Total Confirmados,122
630,Municipio de Huelva sin especificar,Curados,90


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  10396.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2932.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1217 personas en los últimos 7 días 

Un positivo PCR cada 487 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,10396.0,362.0,985.0,510743.0,70.877134,192.856290,101.0
Huelva-Costa,5810.0,249.0,653.0,288115.0,86.423824,226.645610,64.0
Condado-Campiña,3199.0,100.0,286.0,155057.0,64.492412,184.448300,34.0
Huelva (capital),2932.0,118.0,295.0,143663.0,82.136667,205.341668,29.0
Isla Cristina,388.0,46.0,110.0,21264.0,216.328066,517.306245,11.0
Palma del Condado (La),492.0,36.0,67.0,10761.0,334.541399,622.618716,10.0
Lepe,912.0,28.0,85.0,27431.0,102.074296,309.868397,8.0
San Juan del Puerto,147.0,10.0,12.0,9300.0,107.526882,129.032258,7.0
Aljaraque,399.0,22.0,48.0,21260.0,103.480715,225.776105,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Palma del Condado (La),492.0,36.0,67.0,10761.0,334.541399,622.618716,10.0
Isla Cristina,388.0,46.0,110.0,21264.0,216.328066,517.306245,11.0
Lucena del Puerto,83.0,1.0,17.0,3371.0,29.664788,504.301394,0.0
San Bartolomé de la Torre,44.0,3.0,17.0,3736.0,80.299786,455.032120,0.0
Cortelazor,2.0,1.0,1.0,299.0,334.448161,334.448161,0.0
Villalba del Alcor,103.0,3.0,11.0,3338.0,89.874176,329.538646,1.0
Palos de la Frontera,222.0,15.0,35.0,11289.0,132.872708,310.036319,5.0
Lepe,912.0,28.0,85.0,27431.0,102.074296,309.868397,8.0
Cartaya,352.0,23.0,61.0,19974.0,115.149695,305.397016,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lucena del Puerto,83.0,1.0,17.0,3371.0,29.664788,504.301394,0.0,0.058824
Ayamonte,320.0,3.0,26.0,20946.0,14.322544,124.128712,3.0,0.115385
Valverde del Camino,271.0,2.0,12.0,12820.0,15.600624,93.603744,0.0,0.166667
Rociana del Condado,116.0,2.0,12.0,7855.0,25.461489,152.768937,0.0,0.166667
Beas,57.0,1.0,6.0,4257.0,23.490721,140.944327,1.0,0.166667
San Bartolomé de la Torre,44.0,3.0,17.0,3736.0,80.299786,455.032120,0.0,0.176471
Gibraleón,351.0,3.0,16.0,12607.0,23.796304,126.913619,2.0,0.187500
Moguer,339.0,10.0,52.0,22088.0,45.273452,235.421949,2.0,0.192308
Paterna del Campo,61.0,2.0,9.0,3509.0,56.996295,256.483329,1.0,0.222222
